# Data leakage
- 데이터 유출
- 예측할때 사용할수 없는 정보가 학습데이터로 사용되는 것

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
DATA_PATH = "/content/drive/MyDrive/01-python/data/"
SEED = 42

In [3]:
import pandas as pd
import numpy as np

# 변수 설명
- customerID : 고객ID
- gender : 성별(1:여성, 0:남성)
- SeniorCitizen : 고령자여부(1:Yes, 0:No)
- Partner : 기혼여부(1:Yes, 0: No)
- Dependents : 부양가족여부(1:Yes, 0:No)
- tenure : 가입기간(월단위)
- PhoneService : 집전화이용여부(1:Yes, 0:No)
- MultipleLines : 다중회선여부(1:Yes, 0:No)
- InternetService : 인터넷이용방식
- OnlineSecurity : 인터넷보안서비스사용여부(1:Yes, 0:No)
- OnlineBackup : 인터넷백업서비스사용여부(1:Yes, 0:No)
- DeviceProtection : 기기방화벽서비스사용여부(1:Yes, 0:No)
- TechSupport : 인터넷기술지원서비스사용여부(1:Yes, 0:No)
- StreamingTV : 스트리밍TV여부(1:Yes, 0:No)
- StreamingMovies : 스트리밍영화여부(1:Yes, 0:No)
- Contract : 약정기간(연단위, 0은 약정없음)
- PaperlessBilling : 지로여부(1:Yes, 0:No)
- PaymentMethod : 지불방법
- MonthlyCharges : 한달요금
- TotalCharges : 총지불요금
- Churn : 이탈여부(1:Yes, 0:No)

- 학습 및 테스트 데이터 불러오기

In [4]:
train_churn = pd.read_csv(f"{DATA_PATH}train_churn.csv")
test_churn = pd.read_csv(f"{DATA_PATH}test_churn.csv")
train_churn.shape ,test_churn.shape

((5282, 21), (1761, 20))

- 결측치 확인

In [5]:
train_churn.isnull().sum().sum() , test_churn.isnull().sum().sum()

(0, 0)

- 수치형 범주형 변수 확인하기

In [6]:
train_churn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5282 entries, 0 to 5281
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        5282 non-null   object 
 1   gender            5282 non-null   int64  
 2   SeniorCitizen     5282 non-null   int64  
 3   Partner           5282 non-null   int64  
 4   Dependents        5282 non-null   int64  
 5   tenure            5282 non-null   int64  
 6   PhoneService      5282 non-null   int64  
 7   MultipleLines     5282 non-null   int64  
 8   InternetService   5282 non-null   object 
 9   OnlineSecurity    5282 non-null   int64  
 10  OnlineBackup      5282 non-null   int64  
 11  DeviceProtection  5282 non-null   int64  
 12  TechSupport       5282 non-null   int64  
 13  StreamingTV       5282 non-null   int64  
 14  StreamingMovies   5282 non-null   int64  
 15  Contract          5282 non-null   int64  
 16  PaperlessBilling  5282 non-null   int64  


- 학습에 바로 사용할수 있는 수치형 데이터 피쳐로 만들기

In [7]:
train = train_churn.select_dtypes("number").iloc[:,:-1] # 정답값 제외
test = test_churn.select_dtypes("number")

train.shape, test.shape

((5282, 17), (1761, 17))

- 범주형 인코딩 하기 전에 범주의 고유값 개수 확인하기

In [8]:
train_churn.select_dtypes("object").nunique()

customerID         5282
InternetService       3
PaymentMethod         4
dtype: int64

- 학습데이터 먼저 원핫 인코딩

In [9]:
from sklearn.preprocessing import OneHotEncoder
cols = ["InternetService","PaymentMethod"]
enc = OneHotEncoder(handle_unknown="ignore")

tmp = pd.DataFrame(
    enc.fit_transform(train_churn[cols]).toarray(),
    columns = enc.get_feature_names_out()
)

train = pd.concat([train,tmp],axis=1)
train.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,OnlineSecurity,OnlineBackup,DeviceProtection,...,PaperlessBilling,MonthlyCharges,TotalCharges,InternetService_DSL,InternetService_Fiber optic,InternetService_No,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
0,0,0,0,1,1,0,0,0,0,0,...,0,25.30,25.30,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,0,0,0,7,1,0,0,0,1,...,0,75.15,525.00,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,1,0,0,1,4,1,0,0,0,0,...,0,20.05,85.50,0.0,0.0,1.0,1.0,0.0,0.0,0.0
3,1,0,0,0,29,1,1,0,0,0,...,0,76.00,2215.25,0.0,1.0,0.0,0.0,1.0,0.0,0.0
4,1,0,0,0,3,1,1,0,0,0,...,0,75.10,270.70,0.0,1.0,0.0,0.0,1.0,0.0,0.0


- 테스트 데이터에 대해서는 transform 만 해야한다. 
- 절대 fit 하지 말것

In [10]:
tmp = pd.DataFrame(
    enc.transform(test_churn[cols]).toarray(),
    columns = enc.get_feature_names_out()
)

test = pd.concat([test,tmp],axis = 1)
test.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,OnlineSecurity,OnlineBackup,DeviceProtection,...,PaperlessBilling,MonthlyCharges,TotalCharges,InternetService_DSL,InternetService_Fiber optic,InternetService_No,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
0,1,0,1,0,1,0,0,0,0,0,...,0,24.80,24.80,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0,0,0,0,41,1,1,0,0,0,...,0,25.25,996.45,0.0,0.0,1.0,1.0,0.0,0.0,0.0
2,1,0,1,1,52,1,0,0,0,0,...,1,19.35,1031.70,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3,1,0,0,0,1,1,0,0,0,1,...,1,76.35,76.35,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,0,0,0,0,67,1,0,0,0,0,...,1,50.55,3260.10,1.0,0.0,0.0,1.0,0.0,0.0,0.0


# 추가 피쳐 만들어 보기
    - 가입기간별로 여성의 비율을 피쳐로 추가 하기

In [11]:
train_tmp = train_churn.groupby("tenure")["gender"].agg("mean").reset_index()
train_tmp = train_tmp.rename(columns  = {"gender":"gender_rate"})
train_tmp.head()

,tenure,gender_rate
0,0,0.375000
1,1,0.451327
2,2,0.530120
3,3,0.483660
4,4,0.518519


In [12]:
train = train.merge(train_tmp,how="left",on="tenure")

## 테스트 데이터에 대해서 학습데이터에 추가한 동일한 피쳐 추가 시 데이터 유출 사례
- 실제 서비스에서 예측해야 하는 데이터를 하나의 샘플만 예측 해야할때도 있고 여러개의 샘플을 예측해야할때가 있다.
- 다음과 같이 피쳐를 추가 할 경우 하나의 샘플만 예측해야하는경우 피쳐를 만들수 없다.


In [13]:
test_tmp = test_churn.groupby("tenure")["gender"].agg("mean").reset_index()
test_tmp = test_tmp.rename(columns  = {"gender":"gender_rate"})
test.merge(test_tmp,how="left",on="tenure")

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,OnlineSecurity,OnlineBackup,DeviceProtection,...,MonthlyCharges,TotalCharges,InternetService_DSL,InternetService_Fiber optic,InternetService_No,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,gender_rate
0,1,0,1,0,1,0,0,0,0,0,...,24.80,24.80,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.496894
1,0,0,0,0,41,1,1,0,0,0,...,25.25,996.45,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.562500
2,1,0,1,1,52,1,0,0,0,0,...,19.35,1031.70,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.413793
3,1,0,0,0,1,1,0,0,0,1,...,76.35,76.35,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.496894
4,0,0,0,0,67,1,0,0,0,0,...,50.55,3260.10,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.454545
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1756,1,0,0,0,3,1,1,0,0,0,...,75.80,246.30,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.574468
1757,1,0,1,0,8,1,1,0,0,0,...,90.25,743.75,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.541667
1758,0,0,0,0,29,1,1,0,1,1,...,70.90,1964.60,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.388889
1759,1,0,0,0,2,0,0,0,0,0,...,34.70,62.25,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.583333


- 위와 같은 피쳐에 경우 다음과 같이 학습데이터에서 추출한 피쳐를 적용하는 방식으로 해야 데이터 유출을 방지 할수 있다.

In [14]:
test = test.merge(train_tmp,how="left",on="tenure")

In [15]:
train.shape , test.shape

((5282, 25), (1761, 25))

## 데이터 유출이 아닌 예시

- 가입기간 X 한달요금이 총 지불금액 보다 작다면 연체 중일 것이다라고 가정하고 피쳐추가

In [16]:
mask = train["tenure"] * train["MonthlyCharges"] < train["TotalCharges"]
train["add_feature1"] = mask.astype(int) 
train.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,OnlineSecurity,OnlineBackup,DeviceProtection,...,TotalCharges,InternetService_DSL,InternetService_Fiber optic,InternetService_No,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,gender_rate,add_feature1
0,0,0,0,1,1,0,0,0,0,0,...,25.30,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.451327,0
1,1,0,0,0,7,1,0,0,0,1,...,525.00,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.479167,0
2,1,0,0,1,4,1,0,0,0,0,...,85.50,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.518519,1
3,1,0,0,0,29,1,1,0,0,0,...,2215.25,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.462963,1
4,1,0,0,0,3,1,1,0,0,0,...,270.70,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.483660,1


In [17]:
mask = test["tenure"] * test["MonthlyCharges"] < test["TotalCharges"]
test["add_feature1"] = mask.astype(int) 
test.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,OnlineSecurity,OnlineBackup,DeviceProtection,...,TotalCharges,InternetService_DSL,InternetService_Fiber optic,InternetService_No,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,gender_rate,add_feature1
0,1,0,1,0,1,0,0,0,0,0,...,24.80,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.451327,0
1,0,0,0,0,41,1,1,0,0,0,...,996.45,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.629630,0
2,1,0,1,1,52,1,0,0,0,0,...,1031.70,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.450980,1
3,1,0,0,0,1,1,0,0,0,1,...,76.35,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.451327,0
4,0,0,0,0,67,1,0,0,0,0,...,3260.10,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.447368,0


# 백화점 구매기록 데이터 예시

In [18]:
trans_train = pd.read_csv(f"{DATA_PATH}trans_train.csv") # 학습데이터
trans_test = pd.read_csv(f"{DATA_PATH}trans_test.csv")  # 테스트 데이터
trans_train.shape, trans_test.shape

((167465, 7), (64539, 7))

- 학습데이터 피쳐 추출 예시

In [19]:
agg_list = [
        ('총구매액','sum'),
        ('구매건수', 'count'), 
        ('평균구매액', "mean")
    ]

tmp = trans_train.groupby('고객번호')["구매가격"].agg(agg_list).reset_index()
tmp

,고객번호,총구매액,구매건수,평균구매액
0,10004,390166360,68,5.737741e+06
1,10013,22447800,17,1.320459e+06
2,10023,81765810,43,1.901530e+06
3,10040,213705680,66,3.237965e+06
4,10052,358336570,111,3.228257e+06
...,...,...,...,...
2549,49813,47428060,371,1.278384e+05
2550,49892,54369820,121,4.493374e+05
2551,49925,1657400,3,5.524667e+05
2552,49943,40742400,11,3.703855e+06


- 테스트 데이터 피쳐 추출 예시

In [20]:
tmp = trans_test.groupby('고객번호')["구매가격"].agg(agg_list).reset_index()
tmp

,고객번호,총구매액,구매건수,평균구매액
0,10001,3048890,10,3.048890e+05
1,10070,12803700,28,4.572750e+05
2,10075,9546000,7,1.363714e+06
3,10086,4995000,4,1.248750e+06
4,10128,650000,1,6.500000e+05
...,...,...,...,...
941,49903,9178300,44,2.085977e+05
942,49918,160963730,53,3.037052e+06
943,49937,31704700,47,6.745681e+05
944,49949,94106620,101,9.317487e+05


- 백화점구매기록 데이터의 경우 머신러닝에서 학습가능한 상태로 정돈 되어 있지 않다.
- 동일한 고객번호일 경우 하나의 샘플이라고 생각하면 된다.
- groupby 를 이용하여 피쳐 추출시 고객번호를 첫번째 키로 이용한다면 데이터 유출이 아니다.

In [21]:
trans_test.groupby("고객번호").apply(lambda x : x.set_index("고객번호")).head(50)

구매일자    지점    상품번호        중분류     대분류     구매가격
고객번호  고객번호                                                                
10001 10001  2007-07-07 00:00:00   잠실점   47144        굴비류     수산품   220000
      10001  2007-07-07 00:00:00   잠실점  707142        그라스      식기   616320
      10001  2007-07-07 00:00:00   잠실점   38181         건과     농산물    20000
      10001  2007-07-07 00:00:00   잠실점   38101    선식(가루류)     농산물   630000
      10001  2007-10-11 00:00:00   잠실점   50149     일반가공식품    가공식품   216000
      10001  2007-10-11 00:00:00   잠실점   38263      유기농야채     농산물    68850
      10001  2007-10-11 00:00:00   잠실점   37183         야채     농산물    20520
      10001  2007-10-11 00:00:00   잠실점  674157       위생세제    일용잡화    40600
      10001  2007-10-11 00:00:00   잠실점   38181         건과     농산물    19000
      10001  2007-10-11 00:00:00   잠실점   68272  건강식품(비타민)    건강식품  1197600
10070 10070  2007-04-10 00:00:00   강남점   47118        멸치류     수산품   220000
      10070  2007-09-21 00:00:00   강남점   85103      아이스크림    가공식품     5000
      10070  2007-04-10 00:00:00   강남점   47118     상품군미지정      기타    10000
      10070  2007-03-29 00:00:00   강남점  243101     상품군미지정      기타   114000
      10070  2007-03-29 00:00:00   강남점  243101        란제리  란제리/내의  2736000
      10070  2007-10-30 00:00:00   강남점   43106         장류   젓갈/반찬   198000
      10070  2007-10-30 00:00:00   강남점   43106     상품군미지정      기타     9000
      10070  2007-10-22 00:00:00   강남점  821104       한실침구   침구/수예  1113000
      10070  2007-10-01 00:00:00   강남점   88202      아이스크림    가공식품     4000
      10070  2007-10-01 00:00:00   강남점  637166       국산주방    주방용품   483000
      10070  2007-09-21 00:00:00   강남점   85103     상품군미지정      기타    35000
      10070  2007-04-29 00:00:00   강남점  821120        N.B   침구/수예   247000
      10070  2007-09-21 00:00:00   강남점   85103         면류    가공식품    15000
      10070  2007-07-22 00:00:00   강남점   88122      아이스크림    가공식품     6900
      10070  2007-04-10 00:00:00   강남점  243101     상품군미지정      기타   180000
      10070  2007-08-12 00:00:00   강남점   77136     수입단기행사    가공식품     6000
      10070  2007-08-12 00:00:00   강남점   79205       단기행사    일용잡화   216000
      10070  2007-08-12 00:00:00   강남점   39121       산지통합     농산물  1134000
      10070  2007-08-12 00:00:00   강남점   50139     일반가공식품    가공식품    46750
      10070  2007-08-12 00:00:00   강남점   77136       수입식품    차/커피    66000
      10070  2007-09-03 00:00:00   강남점    9112        유제품    축산가공    13750
      10070  2007-09-03 00:00:00   강남점   47118        멸치류     수산품   198000
      10070  2007-09-03 00:00:00   강남점   47118     상품군미지정      기타     9000
      10070  2007-09-03 00:00:00   강남점    8115        유제품    축산가공    22500
      10070  2007-09-21 00:00:00   강남점   85103         밥류    가공식품    30000
      10070  2007-09-21 00:00:00   강남점   85103  즉석.스넥(매장)    가공식품     5000
      10070  2007-04-10 00:00:00   강남점  243101        란제리  란제리/내의  4320000
      10070  2007-09-21 00:00:00   강남점  707142        그라스      식기  1360800
10075 10075  2007-01-18 00:00:00  부산본점  127103     상품군미지정      기타    90000
      10075  2007-01-18 00:00:00  부산본점  127103     기초 화장품     화장품  2160000
      10075  2007-02-03 00:00:00  부산본점  127103     기초 화장품     화장품  2160000
      10075  2007-03-08 00:00:00  부산본점  127103     기초 화장품     화장품  2880000
      10075  2007-03-08 00:00:00  부산본점  127103     상품군미지정      기타   120000
      10075  2007-03-09 00:00:00  부산본점  164128     직수입핸드백    피혁잡화  2046000
      10075  2007-02-03 00:00:00  부산본점  127103     상품군미지정      기타    90000
10086 10086  2007-01-05 00:00:00  본  점  204125       시티웨어    시티웨어  1869000
      10086  2007-01-05 00:00:00  본  점  342205       니  트    시티웨어  2646000
      10086  2007-01-19 00:00:00  본  점  124225   우/양산(특정)    섬유잡화    20000
      10086  2007-01-19 00:00:00  본  점  124225     장갑(특정)    섬유잡화   460000
10128 10128  2007-12-26 00:00:00  영등포점   39132       산지통합     농산물   650000